In [3]:
from mip import Model, xsum, minimize, BINARY

alpha = [1, ]
beta = [0]
qmin = [1/2]
qmax = [1]
Q = 1/2

m = Model()

theta = [m.add_var(var_type=BINARY)]
q = [m.add_var()]

m.objective = minimize(alpha[0] * q[0] + beta[0] - beta[0] * (1 - theta[0]))

#Constraints:
m += q[0] <= qmax[0] * theta[0]
m += q[0] >= qmin[0]* theta[0]
m += q[0] == Q

m.optimize()

q[0].x, theta[0].x

Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 0 (-3) rows, 0 (-2) columns and 0 (-5) elements
Clp0000I Optimal - objective value 0.5
Coin0511I After Postsolve, objective 0.5, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0.5 - 0 iterations time 0.002, Presolve 0.00, Idiot 0.00

Starting MIP optimization


(0.5, 1.0)